# This is Work in Progress (incomplete & undocumented)

# Raster reclassify and export
Given a raster, read and explore it. Then reclassify and export as COG (Cloud Optimized Geotiff) with internal tiling and overviews.

In [ ]:
import rasterio
from rasterio.enums import Resampling
from rasterio.rio.overview import get_maximum_overview_level
from rasterio.plot import show
import matplotlib.pyplot as plt
import pprint
import numpy as np

# Get to know your raster

In [ ]:
dem_r = './inData/demBernCas/dem_bern.tif'
with rasterio.open(dem_r, 'r') as r:
    pprint.pprint(r.profile) # pprint (pretty print) does the same as print, but prettier
    print('-'*30)
    print('Overview levels:', r.overviews(1)) # There is only 1 band, show overviews for it.
    print('-'*30)
    # Plot raster with colorbar, see:
    # https://stackoverflow.com/questions/61327088/rio-plot-show-with-colorbar
    fig, ax = plt.subplots()
    rshow = show(r, ax=ax)
    fig.colorbar(rshow.get_images()[0], ax=ax)

# Read and reclassify

In [ ]:
# Read the raster into memory as nested numpy array.
# Note: If the raster would not fit in memory, we could use a read window in read().
with rasterio.open(dem_r, 'r') as r:
    nodata = r.nodata # Keep track of nodata value for reclassification later
    r_meta = r.meta.copy() # Copy meta data to use later on output raster
    r_data = r.read(1) # Read band 1

# Reclassify the raster
# Use vectorized operations to use full speed potential of numpy.
r_data[r_data==nodata] = 0
r_data[(0 < r_data) & (r_data <= 500)] = 1
r_data[(500 < r_data) & (r_data <= 1000)] = 2
r_data[(1000 < r_data) & (r_data <= 1500)] = 3
r_data[(1500 < r_data) & (r_data <= 2000)] = 4
r_data[r_data > 2000] = 5

# Cast to signed int8 (-127 to 128), because we only deal with a small number of integer categories.
# The resulting file is 25% of the original's size. This might well be premature optimization though.
r_data = r_data.astype('uint8')

# Quick sanity check:
print(f'Min/Max values: {r_data.min()} / {r_data.max()}')

# Save reclassified raster

In [ ]:
out_path = './outData/reclassifiedRaster.tif'

# Use most of the input raster's meta data. Overwrite the parts we changed.
r_meta.update({
    'driver': 'GTiff', # Already there but just to be explicit
    'dtype': 'int8',
    'nodata': 0
})

with rasterio.open(
    out_path, 
    "w", 
    compress='lzw',
    tiled=True,
    **r_meta,
) as dest:
    dest.write(r_data, 1)
    # Create overview factors of power 2 such that the smallest overview width or height 
    # is maximum 256px. As we use discrete categories, use nearest neigbour resampling.
    max_ovr_factor = get_maximum_overview_level(r_meta['width'], r_meta['height'], minsize=256)
    dest.build_overviews([2 ** j for j in range(1, max_ovr_factor+1)], Resampling.nearest)

In [ ]:
# Check meta data of of new raster (crs, overviews, ...)
with rasterio.open(out_path, 'r') as r:
    pprint.pprint(r.profile)
    print('-'*30)
    print('Overview levels:', r.overviews(1))
    print('-'*30)
    fig, ax = plt.subplots()
    rshow = show(r, ax=ax)
    fig.colorbar(rshow.get_images()[0], ax=ax)